<a href="https://colab.research.google.com/github/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/WilliamSteveRodriguezVillamizar_Seminario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Elección de grupos de población homogéneos <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

Elección de grupos de población homogéneos
• Una productora ganadera nos encarga la tarea de seleccionar grupos de
terneros para aplicar 3 tratamientos diferentes. Para cada uno de los
tratamientos debemos seleccionar 3 grupos de terneros separados por sexos
que sean lo mas homogéneos posible en peso para que en los resultados del
tratamiento influya lo menos posible el peso del animal. Disponemos de una
población de N animales entre machos y hembras
• Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la
mejor manera posible las especificaciones de la productora.

....

(*) La respuesta es obligatoria





                                        

In [44]:
!pip install sorting

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [0]:
#Sin ninguna restricción hay 2^n posibles soluciones, donde n es el numero de terneros en el dataset, 
#pero teniendo las restricciones y mirando que el tamaño de datos es limitado en el dataset, 
#y tambien mirando que el sistema solicita grupos homogeneos se generan tambien 2^x posibilidades, 
#donde x es la menor cantidad de inviduos ya sea machos o hembras, siempre y cuando los datos del dataset tengan pesos similares

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

In [0]:
#La estructura mas óptima sea aquella donde los pesos individuales de cada ternero sea muy cercano a la media, 
#puesto que asi, todos serian aptos para usarse en los grupos homogeneos sin descartar a terneros con mucho peso o poco peso, por la restricción que solicita el problema

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

ok - (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [0]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time

tiempoInicial = time()

In [0]:
#Decorador de Ricardo Lebron
from functools import wraps
from time import time

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time() #tomo la hora antes de ejecutar la funcion
        salida = f(*args, **kwargs)
        t_final = time() #tomo la hora despues de ejecutar la funcion
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

In [0]:
nombreArchivo = "TernerosProductoraGanadera.csv"

sexo = {
    'Macho' : 'M',
    'Hembra' : 'H'
}

## Aunque estas tablas nos muestran el precio, tambien podemos usarlas para tener una guia de los pesos conforme a la edad del ternero, esto para tener datos mas realistas


<img src="https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/pesoTerneros.jpg?raw=1">

<img src="https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/pesoTerneros2.jpg?raw=1">

In [0]:
edadRangoPeso = {
    "8" : (120, 160),
    "9" : (160, 180),
    "10" : (180, 200),
    "11" : (200, 230),
    "12" : (230, 260)
}

In [0]:
NombreColumnas = [
    'EtiquetaNacimientoTernero',
    'Sexo',
    'Edad (meses)',
    'Peso'
]

Grupos = {
    "Grupo1" : "A",
    "Grupo2" : "B",
    "Grupo3" : "C"
}

Tratamientos = [
    "1",
    "2",
    "3"
]


Tratamientos_grupos = {
    "Tratamiento1_grupo1" : Tratamientos[0] + Grupos["Grupo1"],
    "Tratamiento1_grupo2" : Tratamientos[0] + Grupos["Grupo2"],
    "Tratamiento1_grupo3" : Tratamientos[0] + Grupos["Grupo3"],
    
    "Tratamiento2_grupo1" : Tratamientos[1] + Grupos["Grupo1"],
    "Tratamiento2_grupo2" : Tratamientos[1] + Grupos["Grupo2"],
    "Tratamiento2_grupo3" : Tratamientos[1] + Grupos["Grupo3"],    
    
    "Tratamiento3_grupo1" : Tratamientos[2] + Grupos["Grupo1"],
    "Tratamiento3_grupo2" : Tratamientos[2] + Grupos["Grupo2"],
    "Tratamiento3_grupo3" : Tratamientos[2] + Grupos["Grupo3"]
}

distanciaMinimaMedia = 10
numero_minimo_terneros_cada_Tratamientos_grupos = 3

In [0]:
import random
def GenerarTerneroAleatorio(etiquetaTernero = None):
    """
    Vamos a generar una edad al azar usando la libreria random, esto debido a que el enunciado dice que es un ternero, pero 
    un ternero se condidera ternero cuando esta en las edades entre 8 y 12 meses, luego se considera novillo,
    y para cada edad hay un intervalo de peso estandarizado, 
    por ejemplo para un ternero de 10 meses su peso suele estar entre 200Kgr y 230Kgr,
    en este orden de ideas, para generar datos lo mas realistas posibles, 
    vamos a generar terneros con pesos que cumplan los intervalos por edades, 
    luego podemos quitar la columna de edad y quedarnos con el genero (generado aleatoriamente) y el peso.
    """
    edad = random.choice(list(edadRangoPeso.keys()) )
    sexoTernero = sexo[random.choice(list(sexo.keys()) )]
    pesoTernero = random.randint(edadRangoPeso[edad][0] , edadRangoPeso[edad][1])
    if sexoTernero == sexo['Macho']:
        #Generalmente los machos son mas pesados
        #Por este motivo a los machos le damos de un 3% a un 10% mas de peso (valor elegido al azar)
        porcentajePeroMayorPorSerMacho = random.randint(3 , 10) #Elegimos el porcentaje de peso que tendra de mas a comparacion si fuere una hembra
        pesoTernero = pesoTernero + (pesoTernero*porcentajePeroMayorPorSerMacho)/100
        
    ternero = { 
        NombreColumnas[0] : etiquetaTernero,
        NombreColumnas[1] : [sexoTernero] ,
        NombreColumnas[2] : [int(edad)], 
        NombreColumnas[3] : [pesoTernero] 
          }
    return ternero

def GenerarGrupoTerneros(sizeDataframe, dataframe = None):
    """
    Usando recursividad vamos a generar el dataframe del tamaño solicitado en la variable "sizeDataframe"
    llenando el dataframe con terneros aleatorios
    """
    ternero = GenerarTerneroAleatorio(sizeDataframe)
    try:
        temporal = pd.DataFrame(ternero)
        dataframe = dataframe.append(temporal)
    except:
        dataframe = pd.DataFrame(ternero)        
    
    if 0 < sizeDataframe:        
        GenerarGrupoTerneros(sizeDataframe -1, dataframe)
    else:
        export_csv = dataframe.to_csv (nombreArchivo, index = None, header=True) 
        
@calcular_tiempo
def GenerarDataframeDatos(sizeDataframe):   
    """
    Colocamos esta funcion para poder medir el tiempo que toma el algoritmo para crear el conjunto de datos 
    (para terminos evaluativos en cuanto a optimizacion del codigo)
    """
    total = numero_minimo_terneros_cada_Tratamientos_grupos * 18
    if numero_minimo_terneros_cada_Tratamientos_grupos > sizeDataframe:
      sizeDataframe = numero_minimo_terneros_cada_Tratamientos_grupos
    GenerarGrupoTerneros(sizeDataframe)

In [52]:
N_animales = 800

#generamos un dataset aleatorio guardado en el archivo "nombreArchivo" declarado al inicio del codigo
GenerarDataframeDatos(N_animales)

Tiempo transcurrido (en segundos): 1.8958361148834229


In [0]:
#cargamos el dataset que hemos guardado en el .csv

@calcular_tiempo
def LeerArchivo():
    return pd.read_csv(nombreArchivo) 

In [54]:
purchases = LeerArchivo()

Tiempo transcurrido (en segundos): 0.007236957550048828


In [0]:
tiempoProcesar = time()

In [0]:
#dado que la columna de edades solo se uso para generar aleatoriamente los pesos
#conforme a la edad, no es necesario que de aca en adelante la sigamos conservando
#por lo cual eliminamos esta columna del dataframe cargado para facilitar el analisis de los datos
purchases = purchases.drop([NombreColumnas[2]], axis=1) 
purchases.dropna(inplace=True) #Borro valores nulos, si llegaren a existir

In [57]:
@calcular_tiempo
def SepararPorGenero(df, genero):
    return df[df[NombreColumnas[1]] == sexo[genero]]

data_male = SepararPorGenero(purchases, "Macho" )

data_female = SepararPorGenero(purchases, "Hembra" )

#dado que ya hemos separado el conjunto de datos por sexos, ya no necesitamos esa columna, 
#por lo cual la podemos eliminar para facilitar el analisis de los datos
data_male = data_male.drop([NombreColumnas[1]], axis=1)
data_female = data_female.drop([NombreColumnas[1]], axis=1) 

Tiempo transcurrido (en segundos): 0.002877473831176758
Tiempo transcurrido (en segundos): 0.0018362998962402344


In [58]:
#ordenamos los conjunto de datos para analizar mas facil la información

@calcular_tiempo
def OrdenarPorPeso(df):
    return df.sort_values(by=[ NombreColumnas[3] ])

data_male = OrdenarPorPeso(data_male)
data_female = OrdenarPorPeso(data_female)

Tiempo transcurrido (en segundos): 0.0024213790893554688
Tiempo transcurrido (en segundos): 0.001402139663696289


In [0]:
@calcular_tiempo
def GruposPorTratamiento(df):
    #print("Tamaño",df.shape[0])    
    #df = df.sort_index()    
    split = int(round(df.shape[0]/3)-1)
    grupoTratamiento = {}
    
    """
    Dividimos el dataframe para tres tratamientos creando 3 grupos
    """
    for i in range(len(Tratamientos)):
        tratamientto = Tratamientos[i]
        grupoTratamiento[tratamientto] = df[:split]
        df = df[split:]
    
    """
    Para cada tratamiento, genero tres grupos, tratando en lo mas posible que mantengan la media en los mismos
    """
    grupos = {}
    for keyTratamiento in grupoTratamiento:
        grupo_para_tratamiento =  grupoTratamiento[keyTratamiento]
        grupo_para_tratamiento = grupo_para_tratamiento.sort_index()
        split = int(round(grupo_para_tratamiento.shape[0]/3)-1)
        #print(keyTratamiento, grupo_para_tratamiento.mean()[1])
        
        for keyGrupo in Grupos:
            llave = Grupos[keyGrupo]
            grupos[keyTratamiento+llave] = grupo_para_tratamiento[:split]
            grupo_para_tratamiento = grupo_para_tratamiento[split:]
        
    return grupos   

In [60]:
"""
Ahora tenermos dos subconjunto de datos (machos y hembras)
Vamos a dividir cada subconjunto en 9, asi tendremos 3 grupos para cada uno de los tres tratamientos
Seguidamente vamos a obtener la media de cada uno de estos 9 grupos y vamos a obtener la distancia de cada elemento del grupo a esta media
Haremos lo mismo tanto para los masculinos como para el conjunto de datos femeninos
"""

grupos_data_male = GruposPorTratamiento(data_male)
grupos_data_female = GruposPorTratamiento(data_female)

Tiempo transcurrido (en segundos): 0.007010221481323242
Tiempo transcurrido (en segundos): 0.0054683685302734375


In [0]:
@calcular_tiempo
def LimpiarOrganizarGruposDatos(df, mediaMinima):
    """
    Cada uno de los nueve grupos, los organizo por cual dato esta mas cercano a la media
    """
    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        dataframe = df[llave]
        MediaDatos = dataframe.mean()[1] #calcular la media de los datos de este grupo

        dataframe['DistanciaMedia'] = abs(MediaDatos - dataframe[NombreColumnas[3]]) #hallo la distancia a la media
        
        busquedaMinimoUno = True
        incrementoRestriccion = 0
        while busquedaMinimoUno:            
            ternero_que_cumple_con_restriccion = dataframe['DistanciaMedia'] <= (mediaMinima + incrementoRestriccion)
            temporal = dataframe[ternero_que_cumple_con_restriccion] #verifico que tenga la distancia minima a la media
            
            if temporal.shape[0] >(numero_minimo_terneros_cada_Tratamientos_grupos - 1):
                dataframe = temporal
                busquedaMinimoUno = False   
            else:
                incrementoRestriccion += 1
        
        dataframe = dataframe.sort_values(by=[ 'DistanciaMedia' ]) #ordeno segun quien este mas cercano a la media
        dataframe = dataframe.drop(['DistanciaMedia'], axis=1)  #quitamos la columna añadida

        df[llave] = dataframe 
    return df

In [62]:
"""
Para cada grupo de cada tratamiento:
Optenemos la media
comparamos cada peso con esta media y hallamos la distancia encomparacion con la media
ordeno los datos conforme a los que tengan menor distancia a la media
"""

grupos_data_male = LimpiarOrganizarGruposDatos(grupos_data_male, distanciaMinimaMedia)
grupos_data_female = LimpiarOrganizarGruposDatos(grupos_data_female, distanciaMinimaMedia)

Tiempo transcurrido (en segundos): 0.713468074798584
Tiempo transcurrido (en segundos): 0.7159955501556396


In [63]:
#Vamos a hallar la menor cantidad para cada grupo, esto para que todos los grupos queden del mismo tamaño

@calcular_tiempo
def determinarDimensionCadaGrupo():
    cantidad_grupo_mas_pequeno = purchases.shape[0]
    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        if cantidad_grupo_mas_pequeno > grupos_data_male[llave].shape[0]:
            cantidad_grupo_mas_pequeno = grupos_data_male[llave].shape[0]

    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        if cantidad_grupo_mas_pequeno > grupos_data_female[llave].shape[0]:
            cantidad_grupo_mas_pequeno = grupos_data_female[llave].shape[0]
    
    return cantidad_grupo_mas_pequeno
            
cantidad_grupo_mas_pequeno = determinarDimensionCadaGrupo()

Tiempo transcurrido (en segundos): 9.131431579589844e-05


In [64]:
@calcular_tiempo
def RecortarGruposDatos(df, corte):
    """
    Cada uno de los nueve grupos, los organizo por cual dato esta mas cercano a la media
    """
    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        dataframe = df[llave]
        dataframe = dataframe[:corte]
        df[llave] = dataframe
    return df

grupos_data_male = RecortarGruposDatos(grupos_data_male, cantidad_grupo_mas_pequeno)
grupos_data_female = RecortarGruposDatos(grupos_data_female, cantidad_grupo_mas_pequeno)

Tiempo transcurrido (en segundos): 0.0048944950103759766
Tiempo transcurrido (en segundos): 0.0047702789306640625


In [0]:
tiempoFinal = time()

In [80]:
conteo = 0
for key in Tratamientos_grupos:     
    llave = Tratamientos_grupos[key]    
    if conteo >= 3:
      conteo = 0
      print("")
    conteo += 1
    print("Male: ", llave, " terneros: " ,grupos_data_male[llave].shape[0],"Media grupo: " , round(grupos_data_male[llave].mean()[1]) )
    
print()
conteo = 0
for key in Tratamientos_grupos:     
    llave = Tratamientos_grupos[key]
    if conteo >= 3:
      conteo = 0
      print("")
    conteo += 1
    print("Female: ", llave, " terneros: " ,grupos_data_female[llave].shape[0] ,"Media grupo: " , round(grupos_data_female[llave].mean()[1]) )

print()
print("Terneros total utilizados: ", cantidad_grupo_mas_pequeno * 18, ", de un total de ", N_animales)

Male:  1A  terneros:  7 Media grupo:  164.0
Male:  1B  terneros:  7 Media grupo:  156.0
Male:  1C  terneros:  7 Media grupo:  160.0

Male:  2A  terneros:  7 Media grupo:  201.0
Male:  2B  terneros:  7 Media grupo:  203.0
Male:  2C  terneros:  7 Media grupo:  200.0

Male:  3A  terneros:  7 Media grupo:  250.0
Male:  3B  terneros:  7 Media grupo:  248.0
Male:  3C  terneros:  7 Media grupo:  249.0

Female:  1A  terneros:  7 Media grupo:  154.0
Female:  1B  terneros:  7 Media grupo:  151.0
Female:  1C  terneros:  7 Media grupo:  155.0

Female:  2A  terneros:  7 Media grupo:  194.0
Female:  2B  terneros:  7 Media grupo:  191.0
Female:  2C  terneros:  7 Media grupo:  192.0

Female:  3A  terneros:  7 Media grupo:  237.0
Female:  3B  terneros:  7 Media grupo:  233.0
Female:  3C  terneros:  7 Media grupo:  233.0

Terneros total utilizados:  126 , de un total de  800


In [67]:
print('Tiempo transcurrido generar dataset (en segundos): {}'.format(tiempoProcesar - tiempoInicial))
print('Tiempo transcurrido generar respuesta problema (en segundos): {}'.format(tiempoFinal - tiempoProcesar))

Tiempo transcurrido generar dataset (en segundos): 2.0550713539123535
Tiempo transcurrido generar respuesta problema (en segundos): 1.6089863777160645


<img src="https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/ternero.jpg?raw=1">

(*)Calcula la complejidad del algoritmo 

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta